<a href="https://colab.research.google.com/github/orlandxrf/curso-dl/blob/main/notebooks/8b_TextGenerationRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación automática de oraciones con una RNN

In [1]:
import torch
from torch import nn

import numpy as np

In [2]:
text = ['hola como te encuentras','estoy muy bien','que tengas un bonito dia']

# une todas las oraciones y extrae los caracteres únicos de las oraciones combinadas
chars = set(''.join(text))

# creación de un diccionario que asigna números a los caracteres
int2char = dict(enumerate(chars))

# crear otro diccionario que asigne caracteres a números
char2int = {char: ind for ind, char in int2char.items()}

In [3]:
print (f"int2char:\t{int2char}")

print (f"char2int:\t{char2int}")

int2char:	{0: 'g', 1: 'c', 2: ' ', 3: 'r', 4: 't', 5: 'b', 6: 'l', 7: 'i', 8: 'h', 9: 's', 10: 'd', 11: 'm', 12: 'q', 13: 'u', 14: 'e', 15: 'a', 16: 'n', 17: 'y', 18: 'o'}
char2int:	{'g': 0, 'c': 1, ' ': 2, 'r': 3, 't': 4, 'b': 5, 'l': 6, 'i': 7, 'h': 8, 's': 9, 'd': 10, 'm': 11, 'q': 12, 'u': 13, 'e': 14, 'a': 15, 'n': 16, 'y': 17, 'o': 18}


In [4]:
# encontrar la longitud de la cadena más larga
maxlen = len(max(text, key=len))

# un ciclo simple que recorre la lista de oraciones y agrega un ' ' espacio en blanco hasta la longitud de la oración
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

In [5]:
# definir listas que van a contener las secuencias de entrada y de objetivo (target)
input_seq = []
target_seq = []

for i in range(len(text)):
  # eliminar el último carácter de la secuencia de entrada
  input_seq.append(text[i][:-1])
    
  # eliminar el primer carácter de la secuencia de destino
  target_seq.append(text[i][1:])
  print(f"\tInput Sequence: {input_seq[i]} longitud: {len(input_seq[i])}\tTarget Sequence: {target_seq[i]} longitud: {len(target_seq[i])}")

print (f"\ninput_seq:\t{input_seq}")
print (f"target_seq:\t{target_seq}")

	Input Sequence: hola como te encuentras longitud: 23	Target Sequence: ola como te encuentras  longitud: 23
	Input Sequence: estoy muy bien          longitud: 23	Target Sequence: stoy muy bien           longitud: 23
	Input Sequence: que tengas un bonito di longitud: 23	Target Sequence: ue tengas un bonito dia longitud: 23

input_seq:	['hola como te encuentras', 'estoy muy bien         ', 'que tengas un bonito di']
target_seq:	['ola como te encuentras ', 'stoy muy bien          ', 'ue tengas un bonito dia']


In [6]:
for i in range(len(text)):
  input_seq[i] = [char2int[character] for character in input_seq[i]]
  target_seq[i] = [char2int[character] for character in target_seq[i]]

print (f"input_seq:\t{input_seq}")
print (f"target_seq:\t{target_seq}")

input_seq:	[[8, 18, 6, 15, 2, 1, 18, 11, 18, 2, 4, 14, 2, 14, 16, 1, 13, 14, 16, 4, 3, 15, 9], [14, 9, 4, 18, 17, 2, 11, 13, 17, 2, 5, 7, 14, 16, 2, 2, 2, 2, 2, 2, 2, 2, 2], [12, 13, 14, 2, 4, 14, 16, 0, 15, 9, 2, 13, 16, 2, 5, 18, 16, 7, 4, 18, 2, 10, 7]]
target_seq:	[[18, 6, 15, 2, 1, 18, 11, 18, 2, 4, 14, 2, 14, 16, 1, 13, 14, 16, 4, 3, 15, 9, 2], [9, 4, 18, 17, 2, 11, 13, 17, 2, 5, 7, 14, 16, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [13, 14, 2, 4, 14, 16, 0, 15, 9, 2, 13, 16, 2, 5, 18, 16, 7, 4, 18, 2, 10, 7, 15]]


In [7]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
  # crear matriz de ceros con la forma de salida deseada
  features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

  # reemplazar el 0 en el índice de carácter relevante con un 1 para representar ese carácter
  for i in range(batch_size):
    for u in range(seq_len):
      features[i, u, sequence[i][u]] = 1
  return features

In [8]:
# input shape --> (batch Size, sequence length, one-hot encoding size)
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)

In [9]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [10]:
# torch.cuda.is_available() comprueba si hay cudas disponibles
is_cuda = torch.cuda.is_available()

# Usar dispositivo de hardware
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, CPU used")

GPU not available, CPU used


In [11]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # definición de algunos parámetros
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # definición de las capas
        # capa RNN
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # capa Fully connected (FC)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)

        # inicializando el estado oculto para la primera entrada
        hidden = self.init_hidden(batch_size)
        
        # Pasar la entrada y el estado oculto al modelo y obtener salidas
        out, hidden = self.rnn(x, hidden)

        # Reshaping de las salidas para que puedan encajar en la capa totalmente conectada
        out = out.contiguous().view(-1, self.hidden_dim)
        
        out = self.fc(out)
      
        return out, hidden
    
    def init_hidden(self, batch_size):
        # este método genera el primer estado oculto de ceros que usaremos en el forward
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [12]:
# crear una instancia del modelo con hiperparámetros
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)

# configurar el modelo para usar el hardware disponible
model.to(device)

n_epochs = 100
lr=0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [13]:
# Entrenamiento
for epoch in range(1, n_epochs + 1):
    # borrar los gradientes existentes de la época anterior
    optimizer.zero_grad()

    input_seq.to(device)
    
    output, hidden = model(input_seq)
    
    loss = criterion(output, target_seq.view(-1).long())
    
    # backpropagation
    loss.backward()
    
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 2.6263
Epoch: 20/100............. Loss: 2.2531
Epoch: 30/100............. Loss: 1.9181
Epoch: 40/100............. Loss: 1.5181
Epoch: 50/100............. Loss: 1.1410
Epoch: 60/100............. Loss: 0.8341
Epoch: 70/100............. Loss: 0.6094
Epoch: 80/100............. Loss: 0.4455
Epoch: 90/100............. Loss: 0.3238
Epoch: 100/100............. Loss: 0.2372


In [14]:
# esta función toma el modelo y el carácter como argumentos y devuelve la predicción del siguiente carácter y el estado oculto
def predict(model, character):
    # codificación one-hot de la entrada para encajar en el modelo
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data

    # tomar la clase con la puntuación de probabilidad más alta de la salida
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [15]:
# esta función toma la longitud de salida deseada y los caracteres de entrada como argumentos, devolviendo la oración producida
def sample(model, out_len, start='hola'):
    model.eval() # evaluar el modelo
    start = start.lower()
    # recorrer a traves de los caracteres iniciales
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # ahora pasar los caracteres anteriores y obtener uno nuevo
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [32]:
sample(model, 20, 'benito')

'benito dia          '

# Ejercicio

Obtener al menos 100 tweets en español sobre un tema específico, puede usarse el #hashtag de algún tema (covid-19, sismo, etc.). Utilizar alguna biblioteca para recolectar tweets

Alimentar la está red y observar los resultados (oraciones):

*   Las oraciones son coherentes?
*   Qué creen se necesitaría para mejorar esto?
*   Para qué sería útil esta red? 

